# World Bank Database
---
## Contract Awards
This set of contract awards includes data on commitments against contracts that were reviewed by the Bank before they were awarded (prior-reviewed Bank-funded contracts) under IDA/IBRD investment projects and related Trust Funds. This dataset does not list all contracts awarded by the Bank, and should be viewed only as a guide to determine the distribution of major contract commitments among the Bank's member countries. "Supplier Country" represents place of supplier registration, which may or not be the supplier's actual country of origin. Information does not include awards to subcontractors nor account for cofinancing. The Procurement Policy and Services Group does not guarantee the data included in this publication and accepts no responsibility whatsoever for any consequences of its use. The World Bank Group complies with all sanctions applicable to World Bank Group transactions.

[world-bank-contracts](https://datacatalog.worldbank.org/dataset/world-bank-contract-awards)

## Projects

The World Bank provides low-interest loans, zero to low-interest credits, and grants to developing countries. These support a wide array of investments in such areas as education, health, public administration, infrastructure, financial and private sector development, agriculture, and environmental and natural resource management. Some of our projects are cofinanced with governments, other multilateral institutions, commercial banks, export credit agencies, and private sector investors.

World Bank Projects & Operations provides access to basic information on all of the World Bank's lending projects from 1947 to the present. The dataset includes basic information such as the project title, task manager, country, project id, sector, themes, commitment amount, product line, and financing. It also provides links to publicly disclosed online documents.

[world-bank-projects](https://datacatalog.worldbank.org/dataset/world-bank-projects-operations)

<br>

### Data Notes
Data are organized around the concept of project with the project id being the key linking the project to related operational datasets such as contract awards, loans/credits/grants, trust funds, etc.



In [1]:
import pymysql
import re
import pandas as pd
import numpy as np
from datetime import datetime

import src
import env

# Acquire

In [2]:
# Load the contracts data
df_contracts = src.get_contract_data()
df_contracts.head()

,As of Date,Fiscal Year,Region,Borrower Country,Borrower Country Code,Project ID,Project Name,Procurement Type,Procurement Category,Procurement Method,...,WB Contract Number,Contract Description,Contract Signing Date,Supplier,Supplier Country,Supplier Country Code,Supplier State,Total Contract Amount (USD),Borrower Contract Reference Number,UN Supplier Flag
0,3/11/2021 0:00,2000,AFE,Angola,AO,P000044,FINANCIAL INSTITUTIO,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,...,1207736,OIL SECTOR STUDY - CONTRCT WITH KPMG,11/20/2000 0:00,KPMG INTERNATIONAL,United Kingdom,GB,Not assigned,800000,KPMG - 11/20/2000,No
1,3/11/2021 0:00,2000,AFE,Angola,AO,P000044,FINANCIAL INSTITUTIO,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,...,1207736,OIL SECTOR STUDY - CONTRCT WITH KPMG,11/20/2000 0:00,KPMG INTERNATIONAL,United Kingdom,GB,Not assigned,800000,KPMG - 11/20/2000,No
2,3/11/2021 0:00,2000,AFE,Madagascar,MG,P052186,MG-Microfinance,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,...,1116602,APPUI AU DEVELOPPEMENT DU RESEAU OTIV (TOAMASINA),9/17/1999 0:00,DEVELOPMENT INTERNA.DESJARDINS,Canada,CA,Not assigned,1964922,1/01/10/99/AGEPMF/DID,No
3,3/11/2021 0:00,2000,AFE,Madagascar,MG,P052186,MG-Microfinance,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,...,1116602,APPUI AU DEVELOPPEMENT DU RESEAU OTIV (TOAMASINA),9/17/1999 0:00,DEVELOPMENT INTERNA.DESJARDINS,Canada,CA,Not assigned,1964922,1/01/10/99/AGEPMF/DID,No
4,3/11/2021 0:00,2000,AFE,South Africa,ZA,P035923,ZA-GEF Cape Penninsula SIL (FY98),Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,...,1116066,"INSTITUTIONAL, LEGAL, POLICY, FINANCIAL, SOCIA...",7/6/1999 0:00,CSIR,South Africa,ZA,Not assigned,178602,WWF-SA-3,No


In [3]:
df_contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246724 entries, 0 to 246723
Data columns (total 22 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   As of Date                          246724 non-null  object
 1   Fiscal Year                         246724 non-null  int64 
 2   Region                              246724 non-null  object
 3   Borrower Country                    246724 non-null  object
 4   Borrower Country Code               246714 non-null  object
 5   Project ID                          246724 non-null  object
 6   Project Name                        246724 non-null  object
 7   Procurement Type                    246724 non-null  object
 8   Procurement Category                246724 non-null  object
 9   Procurement Method                  246724 non-null  object
 10  Product line                        246724 non-null  object
 11  Major Sector                        246

In [4]:
# Load the projects data
df_projects = src.get_project_data()

In [5]:
df_projects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20433 entries, 0 to 20432
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Project ID                         20433 non-null  object 
 1   Region                             20430 non-null  object 
 2   Country                            20430 non-null  object 
 3   Project Status                     20431 non-null  object 
 4   Project Name                       20433 non-null  object 
 5   Project Development Objective      10957 non-null  object 
 6   Implementing Agency                6057 non-null   object 
 7   Consultant Services Required       1230 non-null   object 
 8   Project URL                        20433 non-null  object 
 9   Board Approval Date                18158 non-null  object 
 10  Project Closing Date               16149 non-null  object 
 11  Financing Type                     12700 non-null  obj

# Prepare

## Connect to MySQL Database

In [6]:
cnx, cursor = src.get_cursor(user=env.user, password=env.password)
thread = src.get_connection(user=env.user, password=env.password)

cursor.execute("CREATE DATABASE IF NOT EXISTS worldbank;")
cursor.execute("USE worldbank;")

0

## Commit to MySQL Database

In [7]:
# Write 5 rows of data to the new database to test the connection.
# df_contracts.head().to_sql(name='test',
#                            con = cnx,
#                            if_exists='replace',
#                            index=False)

## Load data from MySQL Database

In [8]:
# Read data from the table.
# pd.read_sql("""SELECT * FROM test;""", thread)

## Database Prep

### Contracts Table Prep
- Clean column names
- Drop columns
- Create seperate tables

In [9]:
cols = list(df_contracts.columns)
df_contracts.columns = [col.replace(' ', '') for col in cols]

In [10]:
df_contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246724 entries, 0 to 246723
Data columns (total 22 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   AsofDate                         246724 non-null  object
 1   FiscalYear                       246724 non-null  int64 
 2   Region                           246724 non-null  object
 3   BorrowerCountry                  246724 non-null  object
 4   BorrowerCountryCode              246714 non-null  object
 5   ProjectID                        246724 non-null  object
 6   ProjectName                      246724 non-null  object
 7   ProcurementType                  246724 non-null  object
 8   ProcurementCategory              246724 non-null  object
 9   ProcurementMethod                246724 non-null  object
 10  Productline                      246724 non-null  object
 11  MajorSector                      246724 non-null  object
 12  WBContractNumber

In [11]:
df_contracts.describe(include='O').T.sort_values(by=['count', 'freq'], ascending=False)

,count,unique,top,freq
AsofDate,246724,1,3/11/2021 0:00,246724
UNSupplierFlag,246724,2,No,244945
SupplierState,246724,51,Not assigned,241333
Productline,246724,18,IBRD/IDA,217662
ProcurementCategory,246724,4,CONSULTANT SERVICES,123814
MajorSector,246724,21,Public Admin,55922
ProcurementMethod,246724,30,International Competitive Bidding,53797
Region,246724,9,LCR,49902
ProcurementType,246724,68,Management /Technical Advice,27525
SupplierCountry,246724,213,India,14040


In [12]:
df_contracts.drop(columns='AsofDate', inplace=True)
df_contracts.ContractSigningDate = pd.to_datetime(df_contracts.ContractSigningDate)
df_contracts.UNSupplierFlag = np.where(df_contracts.UNSupplierFlag == 'No', False, True)

### Projects Table Prep
- Clean column names
- Drop columns
- Create seperate tables

In [13]:
cols = list(df_projects.columns)
df_projects.columns = [col.replace(' ', '') for col in cols]

In [14]:
df_projects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20433 entries, 0 to 20432
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ProjectID                        20433 non-null  object 
 1   Region                           20430 non-null  object 
 2   Country                          20430 non-null  object 
 3   ProjectStatus                    20431 non-null  object 
 4   ProjectName                      20433 non-null  object 
 5   ProjectDevelopmentObjective      10957 non-null  object 
 6   ImplementingAgency               6057 non-null   object 
 7   ConsultantServicesRequired       1230 non-null   object 
 8   ProjectURL                       20433 non-null  object 
 9   BoardApprovalDate                18158 non-null  object 
 10  ProjectClosingDate               16149 non-null  object 
 11  FinancingType                    12700 non-null  object 
 12  CurrentProjectCost

In [15]:
#ProjectID will be the primary key because all values are unique.
df_projects.describe(include='O').T.sort_values(by=['unique', 'freq'], ascending=[True, False])

,count,unique,top,freq
ConsultantServicesRequired,1230,3,Y,739
ProjectStatus,20431,4,Closed,15304
EnvironmentalandSocialRisk,914,4,Substantial,392
EnvironmentalAssessmentCategory,12606,8,B,5730
Region,20430,9,Latin America and Caribbean,3632
LendingInstrument,20199,18,Specific Investment Loan,7668
Theme1,12255,73,Rural services and infrastructure,866
Theme2,9895,73,Rural services and infrastructure,723
Sector3,7276,91,Central Government (Central Agencies),759
Sector2,10729,102,Central Government (Central Agencies),1609


In [16]:
df_projects.ConsultantServicesRequired.value_counts()

Y      739
N      298
TBD    193
Name: ConsultantServicesRequired, dtype: int64

In [17]:
df_projects.ProjectClosingDate = pd.to_datetime(df_projects.ProjectClosingDate).dt.date
df_projects.BoardApprovalDate = pd.to_datetime(df_projects.BoardApprovalDate).dt.date

In [18]:
df_projects.head(1).T

,0
ProjectID,P163945
Region,Africa West
Country,Western Africa
ProjectStatus,Active
ProjectName,Investments towards Resilient Management of GCLME
ProjectDevelopmentObjective,To reduce the vulnerability of coastal areas a...
ImplementingAgency,Sao Tome and Principe - Ministry of Infrastru...
ConsultantServicesRequired,NaN
ProjectURL,http://projects.worldbank.org/P163945/null?lan...
BoardApprovalDate,2021-06-18


In [19]:
# df_contracts.to_csv('.\data\raw\contracts.csv', index=False)
# df_projects.to_csv('.\data\raw\projects.csv', index=False)

# Create Table SCHEMAS

## `contracts` SCHEMA

In [20]:
length_max = []
length_min = []
datatypes = []

for col in df_contracts.columns:
    datatypes.append(df_contracts[col].dtype)

    l_max = df_contracts[col].astype('str').str.len().max()
    l_min = df_contracts[col].astype('str').str.len().min()

    length_max.append(l_max)
    length_min.append(l_min)

pd.DataFrame({'column':df_contracts.columns,
              'dtype':datatypes,
              'min_characters':length_min,
              'max_characters':length_max})

,column,dtype,min_characters,max_characters
0,FiscalYear,int64,4,4
1,Region,object,3,3
2,BorrowerCountry,object,4,15
3,BorrowerCountryCode,object,2,3
4,ProjectID,object,7,7
5,ProjectName,object,3,40
6,ProcurementType,object,12,40
7,ProcurementCategory,object,5,23
8,ProcurementMethod,object,2,40
9,Productline,object,3,20


In [21]:
# contracts SCHEMA

# CREATE TABLE `contracts` (
#   `FiscalYear` year DEFAULT NULL,
#   `Region` char(3),
#   `BorrowerCountry` varchar(15),
#   `BorrowerCountryCode` varchar(3),
#   `ProjectID` char(7),
#   `ProjectName` varchar(40),
#   `ProcurementType` varchar(40),
#   `ProcurementCategory` varchar(23),
#   `ProcurementMethod` varchar(40),
#   `Productline` varchar(20),
#   `MajorSector` varchar(20),
#   `WBContractNumber` int(7) DEFAULT NULL,
#   `ContractDescription` text,
#   `ContractSigningDate date` ,
#   `Supplier` text,
#   `SupplierCountry` varchar(15),
#   `SupplierCountryCode` varchar(3),
#   `SupplierState` varchar(20),
#   `TotalContractAmounts(USD)` bigint DEFAULT NULL,
#   `BorrowerContractReferenceNumber` text,
#   `UNSupplierFlag` bool
#   KEY `ProjectID_idx` (`ProjectID`),
#   CONSTRAINT `ProjectID` FOREIGN KEY (`ProjectID`) REFERENCES `projects` (`ProjectID`)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci

## `projects` SCHEMA

In [22]:
length_max = []
length_min = []
datatypes = []

for col in df_projects.columns:
    datatypes.append(df_projects[col].dtype)

    l_max = df_projects[col].astype('str').str.len().max()
    l_min = df_projects[col].astype('str').str.len().min()

    length_max.append(l_max)
    length_min.append(l_min)

pd.DataFrame({'column':df_projects.columns,
              'dtype':datatypes,
              'min_characters':length_min,
              'max_characters':length_max})

,column,dtype,min_characters,max_characters
0,ProjectID,object,7,7
1,Region,object,3,28
2,Country,object,3,40
3,ProjectStatus,object,3,8
4,ProjectName,object,3,142
5,ProjectDevelopmentObjective,object,1,4000
6,ImplementingAgency,object,3,947
7,ConsultantServicesRequired,object,1,3
8,ProjectURL,object,49,162
9,BoardApprovalDate,object,3,10


In [23]:
# projects SCHEMA

# CREATE TABLE `projects` (
#   `ProjectID` char(7),
#   `Region` text,
#   `Counrtry` text,
#   `ProjectStatus` varchar(8),
#   `ProjectName` text,
#   `ProjectDevelopmentObjective` text,
#   `ImplementingAgency` text,
#   `ConsultantServicesRequired` varchar(3),
#   `ProjectURL` text,
#   `BoardApprovalDate` date,
#   `ProjectClosingDate` date,
#   `FinancingType` text,
#   `CurrentProjectCost` double,
#   `IBRDCommitment` bigint,
#   `IDACommitment` bigint,
#   `TotalIDAandIBRDCommitment` bigint,
#   `GrantAmount` bigint,
#   `Borrower` text,
#   `LendingInstrument` text,
#   `EnvironmentalAssessmentCategory` varchar(3),
#   `EnvironmentalandSocialRisk` varchar(11),
#   `Sector1` text,
#   `Sector2` text,
#   `Sector3` text,
#   `Theme1` text,
#   `Theme2` text,
#   `Theme3` text,
#   PRIMARY KEY (ProjectID)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci

## Create Tables Using SCHEMAS
When creating the new tables for the `worldbank` database, we need to create the `projects` table __first__.

Why?

The `projects` table contains the primary key that connects records from the `contracts` table. SQL has built-in data consistency between tables known as "Referential Integrity". If the primary table is altered, it affects any table that is linked to it by a foreign key.

__INSERTing Data__

From the perspective of the foreign table, the table that contains the primary key is the source of life. You cannot update a foreign table with new keys directly. You must change the source first before you can change the residual.

In [24]:
cursor.execute("""CREATE TABLE IF NOT EXISTS `projects` (
  `ProjectID` char(7) PRIMARY KEY,
  `Region` text,
  `Country` text,
  `ProjectStatus` varchar(8),
  `ProjectName` text,
  `ProjectDevelopmentObjective` text,
  `ImplementingAgency` text,
  `ConsultantServicesRequired` varchar(3),
  `ProjectURL` text,
  `BoardApprovalDate` date,
  `ProjectClosingDate` date,
  `FinancingType` text,
  `CurrentProjectCost` double,
  `IBRDCommitment` bigint,
  `IDACommitment` bigint,
  `TotalIDAandIBRDCommitment` bigint,
  `GrantAmount` bigint,
  `Borrower` text,
  `LendingInstrument` text,
  `EnvironmentalAssessmentCategory` varchar(3),
  `EnvironmentalandSocialRisk` varchar(11),
  `Sector1` text,
  `Sector2` text,
  `Sector3` text,
  `Theme1` text,
  `Theme2` text,
  `Theme3` text
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci""")

0

In [25]:
cursor.execute("""CREATE TABLE IF NOT EXISTS `contracts` (
  `FiscalYear` year DEFAULT NULL,
  `Region` char(3),
  `BorrowerCountry` varchar(15),
  `BorrowerCountryCode` varchar(3),
  `ProjectID` char(7),
  `ProjectName` varchar(40),
  `ProcurementType` varchar(40),
  `ProcurementCategory` varchar(23),
  `ProcurementMethod` varchar(40),
  `Productline` varchar(20),
  `MajorSector` varchar(20),
  `WBContractNumber` int(7) DEFAULT NULL,
  `ContractDescription` text,
  `ContractSigningDate` date,
  `Supplier` text,
  `SupplierCountry` varchar(15),
  `SupplierCountryCode` varchar(3),
  `SupplierState` varchar(20),
  `TotalContractAmount(USD)` bigint DEFAULT NULL,
  `BorrowerContractReferenceNumber` text,
  `UNSupplierFlag` bool,
  KEY `ProjectID_idx` (`ProjectID`),
  CONSTRAINT `ProjectID` FOREIGN KEY (`ProjectID`) REFERENCES `projects` (`ProjectID`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci""")

0

## Insert Data into Tables

In [27]:
# df_projects.to_sql('projects', con=thread, if_exists='append', index=False)
# df_contracts.to_sql('contracts', con=thread, if_exists='append', index=False)

In [28]:
pd.read_sql("""SELECT * FROM projects LIMIT 5;""", con=cnx)

,ProjectID,Region,Country,ProjectStatus,ProjectName,ProjectDevelopmentObjective,ImplementingAgency,ConsultantServicesRequired,ProjectURL,BoardApprovalDate,...,Borrower,LendingInstrument,EnvironmentalAssessmentCategory,EnvironmentalandSocialRisk,Sector1,Sector2,Sector3,Theme1,Theme2,Theme3
0,P000001,Africa,Africa,Closed,West Africa Pilot Community-based Natural Reso...,None,None,None,http://projects.worldbank.org/P000001/west-afr...,1995-09-14,...,None,Specific Investment Loan,B,None,Sub-National Government,Social Protection,"Other Agriculture, Fishing and Forestry",Rural services and infrastructure,Biodiversity,None
1,P000003,Africa,Africa,Closed,REIMP(CEN.ENV.INFO),None,None,None,http://projects.worldbank.org/P000003/reimpcen...,1997-12-18,...,None,Specific Investment Loan,C,None,Other Public Administration,Other Information and Communications Technologies,Social Protection,Land administration and management,Environmental policies and institutions,None
2,P000010,Africa,Africa,Closed,Regional Development Project (03),None,None,None,http://projects.worldbank.org/P000010/null?lan...,1990-02-01,...,None,Financial Intermediary Loan,B,None,Banking Institutions,"Other Agriculture, Fishing and Forestry",Other Transportation,Environmental policies and institutions,Other financial and private sector development,None
3,P000017,Africa,Africa,Closed,Engineering and Technical Assistance Project,None,None,None,http://projects.worldbank.org/P000017/engineer...,1992-05-19,...,None,Technical Assistance Loan,C,None,Power,None,None,Legal institutions for a market economy,Regional integration,None
4,P000031,Africa East,Republic of Angola,Dropped,Manpower Training Project,THE PROJECT WILL (A) PROMOTE SKILLS DEVELOPMEN...,None,None,http://projects.worldbank.org/P000031/manpower...,None,...,None,Specific Investment Loan,C,None,Social Protection,Other Education,None,Other social protection and risk management,None,None


In [29]:
# IT WORKED!!!!!
pd.read_sql("""SELECT * FROM contracts LIMIT 5;""", con=cnx)

,FiscalYear,Region,BorrowerCountry,BorrowerCountryCode,ProjectID,ProjectName,ProcurementType,ProcurementCategory,ProcurementMethod,Productline,...,WBContractNumber,ContractDescription,ContractSigningDate,Supplier,SupplierCountry,SupplierCountryCode,SupplierState,TotalContractAmount(USD),BorrowerContractReferenceNumber,UNSupplierFlag
0,2000,AFE,Angola,AO,P000044,FINANCIAL INSTITUTIO,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,IBRD/IDA,...,1207736,OIL SECTOR STUDY - CONTRCT WITH KPMG,2000-11-20,KPMG INTERNATIONAL,United Kingdom,GB,Not assigned,800000,KPMG - 11/20/2000,0
1,2000,AFE,Angola,AO,P000044,FINANCIAL INSTITUTIO,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,IBRD/IDA,...,1207736,OIL SECTOR STUDY - CONTRCT WITH KPMG,2000-11-20,KPMG INTERNATIONAL,United Kingdom,GB,Not assigned,800000,KPMG - 11/20/2000,0
2,2000,AFE,Madagascar,MG,P052186,MG-Microfinance,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,IBRD/IDA,...,1116602,APPUI AU DEVELOPPEMENT DU RESEAU OTIV (TOAMASINA),1999-09-17,DEVELOPMENT INTERNA.DESJARDINS,Canada,CA,Not assigned,1964922,1/01/10/99/AGEPMF/DID,0
3,2000,AFE,Madagascar,MG,P052186,MG-Microfinance,Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,IBRD/IDA,...,1116602,APPUI AU DEVELOPPEMENT DU RESEAU OTIV (TOAMASINA),1999-09-17,DEVELOPMENT INTERNA.DESJARDINS,Canada,CA,Not assigned,1964922,1/01/10/99/AGEPMF/DID,0
4,2000,AFE,South Africa,ZA,P035923,ZA-GEF Cape Penninsula SIL (FY98),Implementation Activity,CONSULTANT SERVICES,Quality And Cost-Based Selection,GEF,...,1116066,"INSTITUTIONAL, LEGAL, POLICY, FINANCIAL, SOCIA...",1999-07-06,CSIR,South Africa,ZA,Not assigned,178602,WWF-SA-3,0
